In [1]:
import tensorflow as tf

In [2]:
list = range(10)
list

range(0, 10)

In [3]:
dataset = tf.data.Dataset.from_tensor_slices(range(10))
dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [4]:
for item in dataset:
    print(item.numpy())

0
1
2
3
4
5
6
7
8
9


In [5]:
it = iter(dataset)
next(it)

<tf.Tensor: shape=(), dtype=int32, numpy=0>

In [6]:
next(it).numpy()

1

In [7]:
print(dataset.reduce(0, lambda state, value: state + value).numpy())

45


In [8]:
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item.numpy())

[0 1 2 3 4 5 6]
[7 8 9 0 1 2 3]
[4 5 6 7 8 9 0]
[1 2 3 4 5 6 7]
[8 9]


In [9]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5])
dataset = dataset.repeat(3)
for item in dataset:
    print(item.numpy())

1
2
3
4
5
1
2
3
4
5
1
2
3
4
5


In [10]:
dataset = dataset.map(lambda x: x**2)
for item in dataset:
    print(item.numpy())

1
4
9
16
25
1
4
9
16
25
1
4
9
16
25


In [11]:
del list

In [12]:
elements = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
dataset = tf.data.Dataset.from_generator(lambda: elements, tf.int64)

for batch in dataset:
    print(batch.numpy())

[1 2 3]
[4 5 6]
[7 8 9]


In [13]:
dataset = dataset.unbatch()
list(dataset.as_numpy_iterator())

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [14]:
for item in dataset:
    print(item.numpy())

1
2
3
4
5
6
7
8
9


In [15]:
dataset.element_spec

TensorSpec(shape=<unknown>, dtype=tf.int64, name=None)

In [16]:
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random.uniform([4, 10], minval=1, maxval=10, dtype=tf.int32))
dataset1

<TensorSliceDataset element_spec=TensorSpec(shape=(10,), dtype=tf.int32, name=None)>

In [17]:
for item in dataset1:
    print(item.numpy())

[4 8 7 3 3 5 5 2 4 8]
[1 2 1 1 1 8 4 3 7 1]
[5 2 9 8 6 2 6 2 7 8]
[6 9 8 8 9 1 4 3 5 7]


In [18]:
dataset2 = tf.data.Dataset.from_tensor_slices((tf.random.uniform([4]), tf.random.uniform([4, 100], maxval=100, dtype=tf.int32)))
for item in dataset2:
    print(item)

(<tf.Tensor: shape=(), dtype=float32, numpy=0.20005393>, <tf.Tensor: shape=(100,), dtype=int32, numpy=
array([34, 66, 34, 66, 25,  9, 16,  7, 34, 78, 76, 10, 64, 29,  4, 59, 63,
       91, 50, 91, 21, 62, 55, 24, 18, 18,  4, 23,  2, 56, 79, 65, 56, 10,
       79,  4, 73, 41, 29, 32, 94, 85, 21, 53, 57, 87, 38, 28, 31, 73, 16,
       23, 53, 34, 50, 95, 88, 41, 50, 92, 51, 67, 51, 92,  6, 49, 63, 27,
       59, 14, 96, 48, 94, 60, 96, 31, 58, 97, 43, 95, 60, 83, 41, 42, 30,
       42, 12,  2, 19, 60, 97, 23, 35, 38, 41, 50, 29,  9, 89, 39],
      dtype=int32)>)
(<tf.Tensor: shape=(), dtype=float32, numpy=0.5728868>, <tf.Tensor: shape=(100,), dtype=int32, numpy=
array([71, 81,  8,  1,  9, 37,  2,  1, 49, 77, 23, 37, 90, 27, 12, 24, 21,
       70, 83,  8, 17, 99, 51, 70, 34,  9, 52, 25,  4, 35, 71, 62, 66, 25,
       44, 49,  4, 98, 58, 59, 97, 68, 79, 88, 86, 36,  4, 89, 31,  1, 84,
       27, 71,  1, 12, 71, 13, 70, 21, 97, 17, 43, 92, 29, 73, 58, 12, 13,
        7,  5, 33,  9, 42, 83, 

In [19]:
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))

dataset3.element_spec

(TensorSpec(shape=(10,), dtype=tf.int32, name=None),
 (TensorSpec(shape=(), dtype=tf.float32, name=None),
  TensorSpec(shape=(100,), dtype=tf.int32, name=None)))

In [20]:
for a, (b, c) in dataset3:
    print(f"shapes: {a.shape} {b.shape} {c.shape}")

shapes: (10,) () (100,)
shapes: (10,) () (100,)
shapes: (10,) () (100,)
shapes: (10,) () (100,)


### Using with generators

In [21]:
def count(stop):
    i = 0
    while i < stop:
        yield i
        i += 1

In [22]:
for i in count(6):
    print(i)

0
1
2
3
4
5


In [23]:
ds_counter = tf.data.Dataset.from_generator(count, args=[25], output_types=tf.int32, output_shapes=(),)

In [24]:
for batch in ds_counter.batch(10):
    print(batch.numpy())

[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24]


In [25]:
flowers = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)

228827136/228813984 [==============================] - 3s 0us/step


In [26]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_gen = ImageDataGenerator(rescale=1./255, rotation_range=20)
images, labels = next(img_gen.flow_from_directory(flowers))

Found 3670 images belonging to 5 classes.


In [27]:
images.shape

(32, 256, 256, 3)

In [28]:
labels[0]

array([1., 0., 0., 0., 0.], dtype=float32)

In [29]:
dataset = tf.data.Dataset.from_generator(lambda: img_gen.flow_from_directory(flowers), output_types=(tf.float32, tf.float32), 
                                         output_shapes=(images.shape, labels.shape))

In [30]:
for images, labels in dataset.take(1):
    print(images.shape)
    print(labels.shape)

Found 3670 images belonging to 5 classes.
(32, 256, 256, 3)
(32, 5)


In [31]:
x = [1, 2, 3, 4, 5, 6]

dataset = tf.data.Dataset.from_tensor_slices(x)
dataset = dataset.repeat(2).batch(5, drop_remainder=True)

In [32]:
for item in dataset:
    print(item.numpy())

[1 2 3 4 5]
[6 1 2 3 4]


### TFRecord Format

In [33]:
with tf.io.TFRecordWriter('my_data.tfrecord') as f:
    f.write(b"This is my first record.")
    f.write(b"This is my second record.")

In [35]:
filepaths = ['my_data.tfrecord']
dataset = tf.data.TFRecordDataset(filepaths)

for item in dataset:
    print(item.numpy())

b'This is my first record.'
b'This is my second record.'


In [37]:
from tensorflow.train import Feature, Example, BytesList, Features, FloatList, Int64List

person_example = Example(features = Features(feature={
    'name': Feature(bytes_list=BytesList(value=[b'Alice'])), 
    'id': Feature(int64_list = Int64List(value=[123])), 
    'emails': Feature(bytes_list = BytesList(value=[b'akoredeadewole8@gmail.com']))
}))

In [38]:
with tf.io.TFRecordWriter('my_contacts.tfrecord') as f:
    f.write(person_example.SerializeToString())